### 클리닝 후 전처리_서연

#### 데이터 필터링
1. 외국인 유학생 삭제
2. 재외국인 전형 삭제
3. 제외인정자 삭제
4. 12학번까지 삭제 (or 13학번까지 삭제)
5. 신입학/편입학 나누기

In [ ]:
# 외국인 유학생 삭제
merged_df.drop(merged_df[(merged_df["입학전형"]=="제1학기 외국인") | (merged_df["입학전형"]=="제2학기 외국인")].index, inplace=True)
merged_df.drop(["외국인유학생여부"], axis=1, inplace=True)

In [ ]:
# 재외국민과외국인전형 삭제
merged_df.drop(merged_df[merged_df["세부입학전형"]=="제1학기 재외국민과외국인전형"].index, inplace=True)

In [ ]:
# 제외인정자 삭제
merged_df.dropna(subset=["취업유무"], inplace=True)

In [ ]:
# 12학번까지 삭제
merged_df.drop(merged_df[merged_df["입학년도"] < 2013].index, inplace=True)

In [ ]:
# 졸업일 원핫인코딩
merged_df = pd.get_dummies(merged_df, columns = ["졸업일"])

In [ ]:
# 부전공 이수여부 추가
merged_df["부전공"].fillna(0, inplace=True)
merged_df["부전공_이수여부"] = merged_df["부전공"].mask((merged_df["부전공"] != 0), 1)

In [ ]:
## 신입학/편입학 나누기 + 입학구분 삭제
# 신입학
merged_df_new = merged_df[merged_df["입학구분"]=="신입"]
merged_df_new.drop(["입학구분"], axis=1, inplace=True)
# 편입학
merged_df_trans = merged_df[merged_df["입학구분"]=="편입"]
merged_df_trans.drop(["입학구분"], axis=1, inplace=True)

In [ ]:
# 부전공 유무로 변경
student_df["부전공"].fillna(0, inplace=True)
student_df["부전공"].mask((student_df["부전공"] != 0), 1, inplace=True)

----

#### 모델링 직전 추가 전처리

1. 복수전공 여부 추가
2. 입학전형 정시/수시 원핫인코딩
3. 전진탐 전체 수강 횟수로 수정 -> 전진탐_이수횟수
4. 2전공 있는 경우 1전공이랑 평균 → 전공_취득성적
---
- 취업유무 -> 취업여부
- 정시 -> 입학전형
- 2전공_이수여부 -> 복수/연계전공_이수여부

In [ ]:
# 복수전공 이수여부 추가
df["2전공"].fillna(0, inplace=True)
df["2전공_이수여부"] = df["2전공"].mask((df["2전공"] != 0), 1)
df["2전공"].replace(0, None, inplace=True)

In [ ]:
# 잘못된 데이터 수정
df["입학전형"].replace("정시모집(세계지리 오류)", "정시", inplace=True)

# 입학전형 원핫인코딩 정시/수시
df = pd.get_dummies(df, columns = ["입학전형"], dtype=int)
df.rename(columns={"입학전형_정시":"정시", "입학전형_수시":"수시"}, inplace=True)

In [ ]:
# 전진탐 전체 수강 횟수 컬럼 생성
df["전진탐_이수횟수"] = df["전진탐_1학년"] + df["전진탐_2학년"] + df["전진탐_3학년"] + df["전진탐_4학년"]

In [ ]:
# 2전공 있는 경우 1전공이랑 평균 → 전공_취득성적
df["전공_취득성적"] = df[["1전공_취득성적", "2전공_취득성적"]].mean(axis=1, skipna=True)

In [ ]:
# 컬럼명 변경
df.rename(columns={"취업유무":"취업여부", "정시":"입학전형", "2전공_이수여부":"복수/연계전공_이수여부"}, inplace=True)